In [1]:
%%capture
%pip install scrapy requests pandas

In [2]:
import json
import os
from os.path import exists
import pandas as pd
import requests
from tqdm import tqdm

# Setup API for HTTPS requests

### Authentication to use TMDB API

TMDB requires users to create an account to access its API. After creating an account, you can request a bearer token, which is then used to authenticate HTTPS requests. For using this code, the provided _config.json_ file contains the neccessary credentials used in the request header. Finally, the header is parsed with each request.

In [3]:
CONFIG_PATH = SCRAPED_DATA_PATH = os.path.join(os.path.abspath(""), "config.json")
BASE_URL = "https://api.themoviedb.org/3/movie/"
BASE_URL_TOP_RATED = BASE_URL + "top_rated?"

In [4]:
# Load the config file with API credentials
if exists(CONFIG_PATH):
    with open(CONFIG_PATH) as config_file:
        config = json.load(config_file)
        APP_NAME = config["TMDB_APPLICATION_NAME"]
        AUTH_USER = config["TMDB_EMAIL"]
        AUTH_TOKEN = config["TMDB_BEARER_KEY"]
    
    # Define the headers to include the authentication token
    HEADERS = {
        "accept": "application/json",
        "Authorization": f"Bearer {AUTH_TOKEN}",
    }

else:
    print("Config not found!")

In order to check if authentication is valid, the response should return 200:

In [5]:
response = requests.get(BASE_URL_TOP_RATED+"authentication", headers=HEADERS)
print(response)

<Response [200]>


# Scraping TMDB

## Scraping the top 10.000 features on TMDB

The TMDB web structure is page-based, meaning any search performed on their database returns results one page at a time. It is the user's responsibility to specify which page to request. Therefore, to retrieve all search results, it is necessary to determine the total number of pages.

### Finding the number of pages

The total number of pages can be found by using the default base url and access the _"total\_pages"_ field.

In [6]:
response = requests.get(BASE_URL_TOP_RATED, headers=HEADERS)

In [7]:
TOTAL_PAGES = response.json()["total_pages"]
print(f"{TOTAL_PAGES=}")

TOTAL_PAGES=490


### Extracting features from pages

Using the total number of pages, all features can be extracted incrementally for each page and appended to a pandas DataFrame. The DataFrame can then be saved as a .csv file. 
Initially, the DataFrame is set up as follows:

In [8]:
features_df = pd.DataFrame()

Then, using a for-loop, the incemental page-number with the parameters are parsed the HTTPS request via the TMDB API. 
The parameters used in the code return all the highest rated movies from TMDB and sort them by their average rating.
These are then stored as rows in the DataFrame.

In [9]:
# Create the tqdm progress bar
progress_bar = tqdm(range(1, TOTAL_PAGES+1), desc="Scraping TMDB")

for PAGE in progress_bar:

    params = {
        "language": "en-US",
        "page": PAGE,
        "sort_by": "vote_average.desc"
    }

    response = requests.get(
        BASE_URL_TOP_RATED, 
        headers=HEADERS, 
        params=params
    )
    
    respone_json = response.json()

    current_df = pd.json_normalize(
        respone_json, 
        record_path = "results", 
        meta = [
            "page"
        ]
    )

    features_df = pd.concat([features_df, current_df])

Scraping TMDB: 100%|███████████████████████████████████████████| 490/490 [00:24<00:00, 19.62it/s]


### Clean the feature data and reindex the rows

In [10]:
features_clean_df = features_df.drop_duplicates(subset = "id", keep = "first")

features_clean_df = features_clean_df.reset_index()

features_clean_df = features_clean_df.drop("index", axis=1)

In [11]:
features_clean_df = features_clean_df.assign(cast="")

features_clean_df = features_clean_df[[
    "id",
    "title",
    "original_language",
    "overview",
    "cast",
    "popularity",
    "vote_count",
    "vote_average",
    "release_date",
    "genre_ids",
    "poster_path",
    "backdrop_path",
    "adult",
    "page"
]]

features_clean_df.rename(columns={
    "id": "feature_id", 
    "popularity": "feature_popularity"
}, inplace=True)

In [12]:
features_clean_df.iloc[0]

feature_id                                                          278
title                                          The Shawshank Redemption
original_language                                                    en
overview              Imprisoned in the 1940s for the double murder ...
cast                                                                   
feature_popularity                                              210.704
vote_count                                                        27247
vote_average                                                      8.708
release_date                                                 1994-09-23
genre_ids                                                      [18, 80]
poster_path                            /9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg
backdrop_path                          /zfbjgQE1uSd9wiPTX4VzsLi0rGG.jpg
adult                                                             False
page                                                            

## Scraping the cast of features on TMDB 

Using the feature IDs, the cast can be scraped and added to the feature DataFrame. Each cast entry is stored as a list of actors.

In [13]:
cast_df = pd.DataFrame()

# Create the tqdm progress bar
progress_bar = tqdm(features_clean_df.iterrows(), total=len(features_clean_df), desc="Scraping TMDB")

for idx, row in progress_bar:

    params = {
        "language": "en-US",
    }

    movie_id = row["feature_id"]

    BASE_URL_ACTORS = BASE_URL + f"{movie_id}" + "/credits"

    response = requests.get(
        BASE_URL_ACTORS, 
        headers=HEADERS, 
        params=params
    )
    
    respone_json = response.json()

    current_df = pd.json_normalize(
        respone_json, 
        record_path = "cast",
    )
    
    current_df = current_df.assign(feature_id = movie_id)

    cast_df = pd.concat([cast_df, current_df])

Scraping TMDB: 100%|█████████████████████████████████████████| 9635/9635 [14:03<00:00, 11.43it/s]


In [14]:
cast_df = cast_df.astype({'id': int, "cast_id": int,"order": int, "gender": int, "order":int})

In [15]:
cast_df.iloc[0]

adult                                              False
gender                                                 2
id                                                   504
known_for_department                              Acting
name                                         Tim Robbins
original_name                                Tim Robbins
popularity                                        23.975
profile_path            /djLVFETFTvPyVUdrd7aLVykobof.jpg
cast_id                                                3
character                                  Andy Dufresne
credit_id                       52fe4231c3a36847f800b131
order                                                  0
feature_id                                           278
Name: 0, dtype: object

## Update feature data set with actor IDs

By scraping the features cast list, the starring actors can be added to the feature DataFrame as a list, indicated by their TMDB IDs.

In [16]:
# Create the tqdm progress bar
progress_bar = tqdm(cast_df.iterrows(), total=len(cast_df), desc="Adding cast to feature")

for idx, row in progress_bar:
    selected_rows = cast_df[cast_df["feature_id"] == row["feature_id"]]
    selected_rows = selected_rows.sort_values(by="id",ascending = True)
    
    cast = list(selected_rows["id"])
    
    features_clean_df.at[features_clean_df.index[features_clean_df["feature_id"] == row["feature_id"]].item(), "cast"] = cast

Adding cast to feature: 100%|██████████████████████████| 361242/361242 [02:28<00:00, 2435.29it/s]


In [17]:
features_clean_df

,feature_id,title,original_language,overview,cast,feature_popularity,vote_count,vote_average,release_date,genre_ids,poster_path,backdrop_path,adult,page
0,278,The Shawshank Redemption,en,Imprisoned in the 1940s for the double murder ...,"[192, 504, 2141, 2555, 4029, 5063, 6573, 6574,...",210.704,27247,8.708,1994-09-23,"[18, 80]",/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg,/zfbjgQE1uSd9wiPTX4VzsLi0rGG.jpg,False,1
1,238,The Godfather,en,"Spanning the years 1945 to 1955, a chronicle o...","[1158, 1769, 2870, 3084, 3085, 3086, 3087, 308...",182.054,20693,8.689,1972-03-14,"[18, 80]",/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,False,1
2,240,The Godfather Part II,en,In the continuing saga of the Corleone crime f...,"[380, 1004, 1158, 1769, 3085, 3087, 3092, 3093...",113.029,12482,8.572,1974-12-20,"[18, 80]",/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,/kGzFbGhp99zva6oZODW5atUtnqi.jpg,False,1
3,424,Schindler's List,en,The true story of how businessman Oskar Schind...,"[930, 1088, 2282, 2829, 3738, 3896, 3970, 4924...",88.182,15897,8.567,1993-12-15,"[18, 36, 10752]",/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,/zb6fM1CX41D9rF9hdgclu0peUmy.jpg,False,1
4,389,12 Angry Men,en,The defense and the prosecution have rested an...,"[1936, 2651, 3143, 4958, 5247, 5248, 5249, 525...",48.620,8693,8.547,1957-04-10,[18],/ow3wq89wM8qd5X7hWKxiRfsFf9C.jpg,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9630,12142,Alone in the Dark,en,Edward Carnby is a private investigator specia...,"[1234, 2224, 5354, 10822, 10875, 27122, 32205,...",13.258,606,3.246,2005-01-28,"[28, 14, 27]",/bSxrbVCyWW077zhtpuYlo3zgyug.jpg,/lcLyZzhB1ctfdH0hGBsTFrbflqP.jpg,False,490
9631,13805,Disaster Movie,en,"Over the course of one evening, an unsuspectin...","[19754, 20404, 28639, 34202, 47884, 54593, 589...",23.797,1024,3.200,2008-08-29,[35],/3J8XKUfhJiNzwobUZVtizXYPe8b.jpg,/5V6jAFS0Q49SI07qjyFRMYlbfR9.jpg,False,490
9632,11059,House of the Dead,en,"Set on an island off the coast, a techno rave ...","[920, 15661, 32447, 35463, 37696, 43292, 46897...",11.998,386,3.100,2003-04-11,"[27, 28, 53]",/z2mDGbV4pLtsvSMNnmnSgoVZSWK.jpg,/aNUEHLNsNMprLZt6fjf5nqDq6er.jpg,False,490
9633,14164,Dragonball Evolution,en,"On his 18th birthday, Goku receives a mystical...","[503, 1619, 4730, 8874, 9462, 37492, 47297, 53...",16.741,2037,2.891,2009-03-12,"[28, 12, 14, 878, 53]",/sunS9xhPnFNP5wlOWrvbpBteAB.jpg,/oHrrgAPEKpz0S1ofQntiZNrmGrM.jpg,False,490


### Clean the actors data and reindex the rows

In [18]:
cast_clean_df = cast_df[[
    "id",
    "original_name",
    "popularity",
    "gender",
    "adult",
    "profile_path"
]]

cast_clean_df.rename(columns={
    "id": "actor_id", 
    "popularity": "actor_popularity",
    "profile_path": "profile_image_path"
}, inplace=True)

/tmp/ipykernel_91619/746388013.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cast_clean_df.rename(columns={


In [19]:
cast_clean_df = cast_clean_df.sort_values(by = "actor_id", ascending = True)

cast_clean_df = cast_clean_df.drop_duplicates(subset = "actor_id", keep = "first")

cast_clean_df = cast_clean_df.reset_index()

cast_clean_df = cast_clean_df.drop("index", axis=1)

In [20]:
cast_clean_df

,actor_id,original_name,actor_popularity,gender,adult,profile_image_path
0,1,George Lucas,24.366,2,False,/WCSZzWdtPmdRxH9LUCVi2JPCSJ.jpg
1,2,Mark Hamill,22.620,2,False,/2ZulC2Ccq1yv3pemusks6Zlfy2s.jpg
2,3,Harrison Ford,33.902,2,False,/n4dwIg6NbQzeMaS1yEKKlfNJH7a.jpg
3,4,Carrie Fisher,10.695,1,False,/2vmMjXMFW9ebX9VNKPW7WD6HLOR.jpg
4,5,Peter Cushing,10.103,2,False,/if5g03wn6uvHx7F6FxXHLebKc0q.jpg
...,...,...,...,...,...,...
186415,5106470,Michelle Parkman,0.600,0,False,None
186416,5106471,Aja Preston,0.600,0,False,None
186417,5106472,Jackson Stewart,0.600,0,False,None
186418,5106473,Serinity Thomas,0.600,0,False,None


## Saving the scraped feature data to a .csv file

In [21]:
SCRAPED_FEATURES_PATH = os.path.join(os.path.abspath(""), "TMDB_scraped_features.csv")

In [22]:
SCRAPED_FEATURES_PATH

'/home/aleksandar-lukic/school/02805_Final_project/code/TMDB_scraped_features.csv'

In [23]:
features_clean_df.to_csv(SCRAPED_FEATURES_PATH, index = False)

## Saving the scraped actors data to a .csv file

In [24]:
SCRAPED_ACTORS_PATH = os.path.join(os.path.abspath(""), "TMDB_scraped_actors.csv")

In [25]:
SCRAPED_ACTORS_PATH

'/home/aleksandar-lukic/school/02805_Final_project/code/TMDB_scraped_actors.csv'

In [26]:
cast_clean_df.to_csv(SCRAPED_ACTORS_PATH, index = False)

# Scraping Wikipedia via TMDB

# TODO:::NEEDS TO BE UPDATED!!!

# TODO:::FIND ALL WIKIMEDIA\_ITEM AND ADD THEM TO FEATURE_CLEAN_DF!!!

# TODO:::THEN CREATE SEPERATE NOTEBOOK FOR WIKIPEDIA SCRAPING!!!

In [27]:
response_wiki = requests.get("https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&titles=The_Shawshank_Redemption&format=json")

page = list((response_wiki.json()["query"]["pages"]).keys())[0]

response_wiki.json()["query"]["pages"][page]["pageprops"]["wikibase_item"]

movie_id = 278

params = {
        "language": "en-US",
        # "sort_by": "vote_average.desc"
    }

BASE_URL_ACTORS = BASE_URL + f"{movie_id}" + "/external_ids"

response = requests.get(
    BASE_URL_ACTORS, 
    headers=HEADERS, 
    params=params
)


respone_json = response.json()

respone_json

{'id': 278,
 'imdb_id': 'tt0111161',
 'wikidata_id': 'Q172241',
 'facebook_id': None,
 'instagram_id': None,
 'twitter_id': None}

# Downloading feature posters from TMDB 

### Create folder for images

To automize the process of fetching and storing the feature images from TMDB, a folder _"images"_ is created if it doesn't already exist.

In [28]:
IMAGE_FOLDER_PATH = os.path.join(os.path.abspath(""), "images")

In [29]:
if os.path.exists(IMAGE_FOLDER_PATH):
    print(f"Found folder:\n{IMAGE_FOLDER_PATH}")
else:
    os.makedirs(IMAGE_FOLDER_PATH)
    print(f"Created folder:\n{IMAGE_FOLDER_PATH}")

Found folder:
/home/aleksandar-lukic/school/02805_Final_project/code/images


### Iterate over the DataFrame to request the images
 
With the DataFrame complete, the _"backdrop\_path"_ column contains the endpoint for each feature's image backdrop.

By appending these endpoints to the modified base URL, the corresponding .jpg files can be retrieved and stored locally.

In [30]:
def save_feature_image(img, folder_path, feature_id):
    path = os.path.join(folder_path, f"{feature_id}_poster.jpg")
    with open(path, "wb") as f:
        f.write(img.content)

In [31]:
BASE_URL_IMAGE = "https://image.tmdb.org/t/p/original"
HEADERS_IMG = {
    "accept": "application/jpg",
    "Authorization": f"Bearer {AUTH_TOKEN}",
}

In [32]:
# Simply for showing it works.
# Should be updated later!
sample_df = features_clean_df.iloc[0:5]
sample_df

,feature_id,title,original_language,overview,cast,feature_popularity,vote_count,vote_average,release_date,genre_ids,poster_path,backdrop_path,adult,page
0,278,The Shawshank Redemption,en,Imprisoned in the 1940s for the double murder ...,"[192, 504, 2141, 2555, 4029, 5063, 6573, 6574,...",210.704,27247,8.708,1994-09-23,"[18, 80]",/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg,/zfbjgQE1uSd9wiPTX4VzsLi0rGG.jpg,False,1
1,238,The Godfather,en,"Spanning the years 1945 to 1955, a chronicle o...","[1158, 1769, 2870, 3084, 3085, 3086, 3087, 308...",182.054,20693,8.689,1972-03-14,"[18, 80]",/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,False,1
2,240,The Godfather Part II,en,In the continuing saga of the Corleone crime f...,"[380, 1004, 1158, 1769, 3085, 3087, 3092, 3093...",113.029,12482,8.572,1974-12-20,"[18, 80]",/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,/kGzFbGhp99zva6oZODW5atUtnqi.jpg,False,1
3,424,Schindler's List,en,The true story of how businessman Oskar Schind...,"[930, 1088, 2282, 2829, 3738, 3896, 3970, 4924...",88.182,15897,8.567,1993-12-15,"[18, 36, 10752]",/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,/zb6fM1CX41D9rF9hdgclu0peUmy.jpg,False,1
4,389,12 Angry Men,en,The defense and the prosecution have rested an...,"[1936, 2651, 3143, 4958, 5247, 5248, 5249, 525...",48.620,8693,8.547,1957-04-10,[18],/ow3wq89wM8qd5X7hWKxiRfsFf9C.jpg,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,False,1


In [34]:
# Create the tqdm progress bar
progress_bar = tqdm(sample_df.iterrows(), total=len(sample_df), desc="Saving posters")

for idx, row in progress_bar:
                
        # get id and backdrop endpoint
        feature_id = row["feature_id"]
        feature_backdrop_path = row["poster_path"]
        
        
        # Update progress bar with current id
        progress_bar.set_postfix(current_id=feature_id)
        
        # Send HTTPS GET request to retrieve the image and then save it to folder
        img = requests.get(BASE_URL_IMAGE + feature_backdrop_path, headers = HEADERS_IMG)
        save_feature_image(img, IMAGE_FOLDER_PATH, feature_id)

Saving posters: 100%|██████████████████████████████| 5/5 [00:00<00:00,  7.58it/s, current_id=389]
